In [82]:
# Read in dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

In [83]:
# Create an engine to a SQLite database called 'traffic.sqlite'
engine = create_engine("sqlite:///traffic.sqlite")

In [84]:
# Create a connection to the engine
conn = engine.connect()

## Creating Table

In [85]:
# Use declarative_base to model the measurements 
Base = declarative_base()

class Incidents(Base):
    __tablename__ = 'incidents'
    
    id = Column(Integer, primary_key=True)
    address = Column(Text)
    issue_reported = Column(Text)
    latitude = Column(Numeric)
    longitude = Column(Numeric)
    published_date = Column(Text)

In [86]:
# Create the Reviews table in the database
Base.metadata.create_all(engine)

In [87]:
# Load the cleaned csv file
df = pd.read_csv("cleanData.csv", index_col=0, encoding='utf-8')
df.head()

,address,issue_reported,location_latitude,location_longitude,published_date
0,15436 Fm 1825 Rd,Crash Urgent,30.447823,-97.662267,2018-02-05
1,N Mopac Expy Svrd Sb & W Anderson Ln,Traffic Hazard,30.361209,-97.745268,2017-12-01
2,W William Cannon Dr & Brodie Ln,Crash Service,30.213794,-97.830349,2018-01-23
3,1800 E Stassney Ln,Traffic Hazard,30.196998,-97.760701,2018-01-24
4,2453 E SH 71 SVRD WB,Crash Service,30.213473,-97.659149,2017-12-15


In [88]:
# Change wine_df to a dictionary
# orient='records' makes the dictionary list-like [{column -> value}]
data_incidents = df.to_dict(orient='records')

In [89]:
data_incidents[0]

{'address': '15436 Fm 1825 Rd',
 'issue_reported': 'Crash Urgent',
 'location_latitude': 30.447823,
 'location_longitude': -97.662267,
 'published_date': '2018-02-05'}

In [90]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [91]:
# Save the referenct to the 'reviews' table
incidents_table = sqlalchemy.Table('incidents', metadata, autoload=True)

In [92]:
# Delete any pre-existing table
# DO NOT USE STEP IN PRODUCTION
conn.execute(incidents_table.delete())

In [93]:
# Insert data into the table
conn.execute(incidents_table.insert(), data_incidents)

In [94]:
# Test the first 5 rows
conn.execute('SELECT * FROM incidents LIMIT 5').fetchall()

[(1, '15436 Fm 1825 Rd', 'Crash Urgent', None, None, '2018-02-05'),
 (2, 'N Mopac Expy Svrd Sb & W Anderson Ln', 'Traffic Hazard', None, None, '2017-12-01'),
 (3, 'W William Cannon Dr & Brodie Ln', 'Crash Service', None, None, '2018-01-23'),
 (4, '1800 E Stassney Ln', 'Traffic Hazard', None, None, '2018-01-24'),
 (5, '2453 E SH 71 SVRD WB', 'Crash Service', None, None, '2017-12-15')]